In [1]:
import numpy as np
import skimage.transform as skt
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import skimage.transform as skt
import my_alg
import tensorflow as tf
from tqdm import tqdm
from ipyfilechooser import FileChooser
from skimage.color import rgba2rgb


2023-02-25 19:51:13.078842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
def path_changed(picker):
    global ccm_fc
    global ref_dir
    global ccm_dir
    if picker == ccm_fc:
        ccm_dir = picker.selected_path
    else:
        ref_dir = picker.selected_path

In [36]:
#get the ccm and ref img directories
global ccm_fc
global ref_fc
ccm_fc = FileChooser(title='Choose the CCM directory', multiple=False, default_path=os.path.expanduser('~'))
ref_fc = FileChooser(title='Choose the reference image directory', multiple=False, default_path=os.path.expanduser('~'))
ccm_fc.register_callback(path_changed)
ref_fc.register_callback(path_changed)
display(ccm_fc)
display(ref_fc)

FileChooser(path='/home/jovyan/work/Spring/computational_photography/CCM_project/U_net', filename='', title='C…

FileChooser(path='/home/jovyan/work/Spring/computational_photography/CCM_project/U_net', filename='', title='C…

In [37]:
img_ext = ('.jpg', '.png', '.bmp', '.tif')
if ref_dir is None:
    print('No reference image directory selected. Defaulting to home directory')
    ref_dir = os.path.expanduser('~')

if ccm_dir is None:
    print('No ccm directory selected. Defaulting to home directory')
    ccm_dir = os.path.expanduser('~')
    
if (len([f for f in os.listdir(ccm_dir) if f.endswith == '.npy']) > 0):
    print('Warning: The ccm direction already contains .npy files. They will be overwritten.')

if (len([f for f in os.listdir(ref_dir) if f.endswith == '.npy']) > 0):
    print('Warning: The reference image direction already contains .npy files. They will be overwritten.')

# get the list of image files in the ccm and ref directories
ccm_file_lst = [f for f in os.listdir(ccm_dir) if f.endswith(img_ext)]
gt_file_lst = [f for f in os.listdir(ref_dir) if f.endswith(img_ext)]
# if there are no files in the ccm directory, exit
if len(ccm_file_lst) == 0:
    print('No image files in the ccm directory')
    exit(1)

# if the number of files in the ccm and ref directories are not the same, exit
# as this is not a valid dataset
if len(ccm_file_lst) != len(gt_file_lst):
    print('The number of files in the ccm and ref directories are not the same')
    exit(1)


In [40]:
ref_imgs = []
ref_imgs1 = []
ccm_imgs = []
ccm_imgs1 = []
reshape_shape = [128,128] #[128,128,3]

# loop over all images in file_lst
i = 0

for name in tqdm(ccm_file_lst):
    i += 1
    # read and process the ref image
    ref_img_path = os.path.join( ref_dir, name)
    ref_img = plt.imread(ref_img_path)
    ref_img = ref_img - np.min(ref_img)
    ref_img = ref_img / np.max(ref_img)
    ref_img = skt.resize(ref_img, reshape_shape)
    # If there is an alpha channel, remove it.
    ref_img = ref_img[:,:,:3]
    ref_imgs.append( ref_img)
    # read and process the ccm image
    # assume that the ccm image has the same name as the ref image
    ccm_img_path = os.path.join( ccm_dir, name)
    ccm_img = plt.imread(ccm_img_path)
    ccm_img = ccm_img - np.min(ccm_img)
    ccm_img = ccm_img / np.max(ccm_img)
    ccm_img = skt.resize(ccm_img, reshape_shape)
    ccm_imgs.append(ccm_img)
    
ref_imgs = np.array( ref_imgs)
ccm_imgs = np.array( ccm_imgs)

100%|██████████| 2/2 [00:00<00:00,  8.92it/s]


In [41]:
#save the processed dataset: You need to adjust the directory.
np.save( f'{ccm_dir}/ccm_optrode_flow_128_allnorm_ccm1_128.npy',ccm_imgs)
np.save( f'{ref_dir}/ref_optrode_flow_128_allnorm_ccm1_128.npy',ref_imgs)

In [5]:
## Test cell for debugging and such
reshape_shape = [128,128] #[128,128,3]
ref_img_path = '/home/jovyan/phots/Photos/1.png'
ref_img = plt.imread(ref_img_path)
ref_img = ref_img - np.min(ref_img)
ref_img = ref_img / np.max(ref_img)
ref_img = skt.resize(ref_img, reshape_shape)


array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)